In [17]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import progressbar
import random
import seaborn as sns
import time

from IPython.display import clear_output
from Diffusion.Model       import *
from Algorithms.Heuristics import *

sns.set_style("darkgrid")

In [20]:
algorithms = [random_solution, degree_solution]
n_trials = 10
seed_set_sizes = list(range(0, 1000+1, 100))
seed_set_sizes[0] = 1
time_horizon = 100

columns = ["trial", "opinion", "activated", "algorithm", "seed_size", "alg-runtime"]
data = {col: [] for col in columns}

In [22]:
i = 0
pbar = progressbar.ProgressBar(maxval=n_trials*len(seed_set_sizes)).start()
for trial in range(n_trials):
    graph = nx.barabasi_albert_graph(10000, 100)
    opinion = np.array([random.random() for user in graph.nodes()])
    ffm = {user: {factor: random.random() for factor in "OCEAN"} for user in graph.nodes()}
    model = BIC(graph, ffm, opinion)
    model.prepare()

    for n_seeds in seed_set_sizes:
        for algorithm in algorithms:
            start_time = time.time()
            active_set = algorithm(model, n_seeds)
            killed_set = set()
            runtime = time.time() - start_time

            for t_step in range(time_horizon+1):
                active_set, killed_set = model.diffuse(active_set, killed_set, t_step)

            data["trial"].append(trial)
            data["opinion"].append(model.total_opinion())
            data["activated"].append(len(active_set.union(killed_set)))
            data["algorithm"].append(algorithm.__name__)
            data["seed_size"].append(n_seeds)
            data["alg-runtime"].append(runtime)

            # Update the progressbar.
            i += 1
            pbar.update(i)
            clear_output(wait=True)

0% |                                                                        |

TypeError: unsupported operand type(s) for -: 'set' and 'list'